In [ ]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json
import pandas as pd
import numpy as np
import twitter_credentials

In [ ]:
consumer_key=twitter_credentials.CONSUMER_KEY
consumer_secret=twitter_credentials.CONSUMER_SECRET
access_token=twitter_credentials.ACCESS_TOKEN
access_secret=twitter_credentials.ACCESS_TOKEN_SECRET

In [ ]:
class TweetsListener(StreamListener):
    def __init__(self,csocket):
        self.client_socket=csocket
    def on_data(self,data):
        try:
            msg=json.loads(data)
            df=pd.DataFrame(data=[msg['text']],columns=['tweets'])
            df['id'] = np.array([msg['id']])
            df['len'] = np.array([len(msg['text'])])
            df['date'] = np.array([msg['created_at']])
            df['source'] = np.array([msg['source']])
            df['likes'] = np.array([msg['favorite_count']])
            df['retweets'] = np.array([msg['retweet_count']])
            print(df)
            df.to_csv('tweet.csv', mode='a', header=False)
            self.client_socket.send(df)
            return True
        except BaseException as e:
            print("error on_data:%s" % str(e))
        return True
    def on_error(self,status):
        print(status)
        return True


In [ ]:
def sendData(c_socket):
    auth=OAuthHandler(consumer_key,consumer_secret)
    auth.set_access_token(access_token,access_secret)
    twitter_stream=Stream(auth,TweetsListener(c_socket))
    twitter_stream.filter(track=['narendramodi'])

In [ ]:
s=socket.socket()
host="127.0.0.1"
port=7918
s.bind((host,port))
print("listening on port %s" % str(port))


In [ ]:
s.listen(5)
c,addr=s.accept()
print("Recieved request from "+ str(addr))

In [ ]:
sendData(c)